In [2]:
from diagrams import Diagram, Cluster, Edge
from diagrams.onprem.workflow import Airflow
from diagrams.aws.storage import S3
from diagrams.programming.language import Python
from diagrams.custom import Custom

# Define the visual context
graph_attr = {
    "fontsize": "10",       # Font size for the text
    "size": "10,10",        # Diagram width and height (inches)
    "nodesep": "0.3",       # Reduce spacing between nodes
    "ranksep": "0.5",       # Reduce vertical spacing between levels
    "dpi": "250"            # Dots per inch (increase this to improve resolution)
}

filename = "flow_diagram"

with Diagram("Airflow Data Pipeline with FastAPI", show=False, direction="LR", graph_attr=graph_attr, filename=filename):
    # Data Ingestion with Airflow
    with Cluster("Data Ingestion Pipeline"):
        airflow = Airflow("Airflow Trigger")
        scraper = Python("Scraper")
        storage_s3 = S3("S3: PDF & Images")
        structured_data_snowflake = Custom("Snowflake: Structured Data", "/Users/nishitamatlani/Documents/Assignment_3/Architecture Diagram/images/snowflake.png")

        # Workflow connections in the Data Ingestion Pipeline
        airflow >> Edge(label="Trigger Scraping", minlen="2") >> scraper
        scraper >> Edge(label="Store PDFs & Images", minlen="2") >> storage_s3
        scraper >> Edge(label="Store Structured Data", minlen="2") >> structured_data_snowflake
        storage_s3 >> Edge(label="Pdf/Image Links") >> structured_data_snowflake

    # Client-Facing Application Cluster with optimized layout
    with Cluster("Client-Facing Application", graph_attr={"pad": "1"}):
        fastapi_service = Custom("FastAPI", "/Users/nishitamatlani/Documents/Assignment_3/Architecture Diagram/images/FastAPI.png")
        nvidia_nemo = Custom("NVIDIA NeMo (NIM)", "/Users/nishitamatlani/Documents/Assignment_3/Architecture Diagram/images/nvidia.png")
        streamlit_app = Custom("Streamlit UI", "/Users/nishitamatlani/Documents/Assignment_3/Architecture Diagram/images/Streamlit.png")
        
        # Connections with optimized alignment
        storage_s3 >> Edge(label="Provide Access to Files", minlen="2") >> fastapi_service
        fastapi_service >> Edge(label="Generate Summaries, Q/A", minlen="2") >> nvidia_nemo
        nvidia_nemo >> Edge(label="Process and Respond", minlen="2") >> fastapi_service
        fastapi_service >> Edge(label="Serve Data", minlen="2") >> streamlit_app
